In [148]:
import pandas as pd
import numpy as np
import os
import json
from os.path import exists 

In [153]:
route = '../../Databases/songs/SpotifyMP3/SongsInfoFirstBatch.json'
f = open(route,)
listSongsFull = json.load(f)
f.close()

In [154]:
baseroute = '../../Databases/result/'
listSongs = []
for song in listSongsFull:
    listSongs.append(song['id']+'.csv')

In [151]:
len(listSongs)

2501

In [155]:
chordsSongs = []
#Read all the csv from the list and find out the key
for i,song in enumerate(listSongs):
    if exists(baseroute+song):
        df = pd.read_csv(baseroute + song, header=None)
    #Get key from song
        key =  df.value_counts().index[0][0]
        chords = df[0].values.tolist()
        chordsSongs.append((i,key, chords))

In [125]:
chordsSongs[0]

(0,
 'C#m',
 ['N',
  'C#',
  'C#aug',
  'C#',
  'C#aug',
  'C#m',
  'A',
  'C#m6',
  'C#aug',
  'C#m',
  'A',
  'F#',
  'C#m',
  'C#aug',
  'C#',
  'B6',
  'C#m',
  'Amaj7',
  'E6',
  'B6',
  'Bbm7b5',
  'Amaj7',
  'F#m7',
  'A',
  'F#m',
  'C#m',
  'Emaj7',
  'F#m7',
  'A',
  'F#m',
  'C#m',
  'Abmaj7',
  'C#m',
  'C#aug',
  'C#m',
  'F#',
  'C#m6',
  'A',
  'C#m',
  'Abm',
  'C#m',
  'Amaj7',
  'E6',
  'Bbm7b5',
  'Amaj7',
  'F#m7',
  'A',
  'F#m',
  'C#m',
  'E6',
  'F#m7',
  'A',
  'F#m',
  'C#m',
  'Abmaj7',
  'Ab',
  'C#',
  'C#m',
  'N'])

In [208]:
#STR Turn b chords to Sharp 
def to_Sharp(chord):
    mapCleanedChords = {
        'Ab': 'G#',
        'Abm': 'G#m',
        'Abmb': 'G#m',
        'Amb': 'G#m',
        'Abdim': 'G#dim',
        'Amb': 'Am',
        'Bmb': 'Bm',
        'Bb': 'A#',
        'Bbm': 'A#m',
        'Bbmb': 'A#m',
        'Bbdim': 'A#dim',
        'C#mb': 'C#m',
        'Cmb': 'Cm',
        'Db': 'C#',
        'Dbm': 'C#m',
        'Dmb': 'C#m',
        'D#mb': 'D#m',
        'Dmb': 'Dm',
        'Dbdim': 'C#dim',
        'Emb': 'D#m',
        'Eb': 'D#',
        'Emb': 'Em',
        'Ebm': 'D#m',
        'Ebmb': 'D#m',
        'Ebdim': 'D#dim',
        'Fmb': 'Fm',
        'F#mb': 'F#m',
        'Gb': 'F#',
        'Gmb': 'Gm',
        'Gbm': 'F#m',
        'Gmb': 'F#m',
        'Gbdim': 'F#dim',
    }
    return mapCleanedChords[chord]
def keyToAnalyze(key):
    return key.replace('m','')

#STR Remove all non-accepted chars from a chord
def remove_chars(chord):
    strChord = chord
    strChord = strChord.replace('7', '')
    strChord = strChord.replace('6', '')
    strChord = strChord.replace('5', '')
    strChord = strChord.replace('4', '')
    strChord = strChord.replace('maj', '')
    strChord = strChord.replace('aug', '')    
    strChord = strChord.split('/', 1)[0] if '/' in strChord else strChord
    strChord = to_Sharp(strChord) if 'b' in strChord else strChord
    return strChord

#[] Remove "N"
def remove_nan(chords):
    chordsnotNAN = list(filter(lambda x: x != 'N', chords))
    return chordsnotNAN

#[],[],str,str TRANSPOSE C
def transpose_C(chords, song_id, comp_key):
    comp_key = keyToAnalyze(comp_key)
    print("comp Key After f(x)", comp_key)
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    chords = remove_nan(chords)
    chords = [remove_chars(chord) for chord in chords]
    new_chords = [""]*len(chords)  # Empty list for new chords
    if comp_key == 'C':
        return({'song_id': song_id,
                'orig_key': comp_key,
                'trans_chords': chords})
    else:
        if comp_key not in notes:
            print('TRANSPOSE COMP KEY: ',comp_key)
            comp_key = max(set(chords), key=chords.count)
            comp_key = keyToAnalyze(comp_key)
            print("NEW PROPOSED: ", comp_key)
        #How many towards "C"
        steps = np.where(np.asarray(notes) == comp_key)[0][0]
        for i in range(len(chords)):
            is_min = False
            is_dim = False
            curr_chord = chords[i]
            if 'm' in curr_chord:
                if "dim" in curr_chord:
                    is_dim = True
                    curr_chord = curr_chord.replace("dim","")
                else:
                    is_min = True
                    curr_chord = curr_chord.replace("m","")
            chord_idx = np.where(np.asarray(notes) == curr_chord)[0][0]
            new_chord_idx = chord_idx - steps
            if is_min:
                new_chords[i] = notes[new_chord_idx] + 'm'
            elif is_dim:
                new_chords[i] = notes[new_chord_idx] + 'dim'
            else:
                new_chords[i] = notes[new_chord_idx]
        return({'song_id': song_id,
                'orig_key': comp_key,
                'trans_chords': new_chords})

#CLEAN CHORDS
def get_clean_chord_progressions(Chords, Id, Key):
    songs = transpose_C(Chords, Id, Key)
    return(songs)

#GET INFO FROM SPECIFIC SONG
def get_song(idx):
    Chords = chordsSongs[idx][2]
    Id = chordsSongs[idx][0]
    Key = remove_chars(chordsSongs[idx][1])
    print(Key)
    return (Chords, Id, Key)


#MAIN FUNCTION GET ALL
def transform_whole(idx):
    Chords,Id,Key =  get_song(idx)
    result = get_clean_chord_progressions(Chords ,Id, Key)
    songInfo = result
    return(songInfo)

def transform_all(info):
    results =[]
    for idx in range(len(info)):
        results.append(transform_whole(idx))
    return results

In [209]:
chordsSongsCleaned = transform_all(chordsSongs)

C#m
comp Key After f(x) C#
D#
comp Key After f(x) D#
Am
comp Key After f(x) A
C#m
comp Key After f(x) C#
Em
comp Key After f(x) E
E
comp Key After f(x) E
C
comp Key After f(x) C
Bm
comp Key After f(x) B
E
comp Key After f(x) E
A#m
comp Key After f(x) A#
Em
comp Key After f(x) E
Fm
comp Key After f(x) F
Gm
comp Key After f(x) G
F
comp Key After f(x) F
F
comp Key After f(x) F
Cm
comp Key After f(x) C
A#
comp Key After f(x) A#
F#m
comp Key After f(x) F#
A#m
comp Key After f(x) A#
D#
comp Key After f(x) D#
A#
comp Key After f(x) A#
Dm
comp Key After f(x) D
F#
comp Key After f(x) F#
C#
comp Key After f(x) C#
C
comp Key After f(x) C
A#m
comp Key After f(x) A#
A#
comp Key After f(x) A#
Em
comp Key After f(x) E
G#
comp Key After f(x) G#
C#
comp Key After f(x) C#
Bm
comp Key After f(x) B
Cm
comp Key After f(x) C
E
comp Key After f(x) E
A
comp Key After f(x) A
C
comp Key After f(x) C
C
comp Key After f(x) C
D
comp Key After f(x) D
A#m
comp Key After f(x) A#
E
comp Key After f(x) E
E
comp Key Aft

In [211]:
len(chordsSongsCleaned)

2500

In [214]:

index = 6000000
matrix = []
for i, chords in enumerate(chordsSongsCleaned):
    index += 1
    matrix.append([index, listSongs[i], ','.join(chords['trans_chords']), chords['orig_key']])
table = pd.DataFrame(matrix, columns=['id_chord', 'id_track', 'chords', 'orig_key'])        
table

os.makedirs('../../Experiments/data', exist_ok=True)  
table.to_csv('../../Experiments/data/batchSongs.csv')